In [1]:
import pandas as pd
import numpy as np

### 读取hs300数据

In [3]:
hs300 = pd.read_csv("01_data/hs300.csv")
hs300

,交易日期,开盘点位,最高点位,最低点位,收盘价,成交量(万股),成交额(万),涨跌幅(%)
0,2007-01-04,2073.25,2139.49,2054.24,2067.09,1067197.07,8238107.26,1.28
1,2007-01-05,2051.15,2083.40,2030.76,2072.88,927434.47,6802606.02,0.28
2,2007-01-08,2072.01,2131.74,2071.72,2131.56,847679.09,6630393.61,2.83
3,2007-01-09,2137.49,2201.36,2128.06,2200.09,815776.29,6545619.27,3.22
4,2007-01-10,2210.76,2255.97,2194.77,2255.97,849049.79,6882262.48,2.54
...,...,...,...,...,...,...,...,...
4128,2023-12-25,3330.33,3348.88,3330.33,3347.45,745426.93,12183832.71,0.31
4129,2023-12-26,3345.40,3345.42,3315.94,3324.79,747329.35,12168999.74,-0.68
4130,2023-12-27,3326.93,3341.27,3309.67,3336.36,944039.28,14092492.37,0.35
4131,2023-12-28,3335.56,3423.40,3331.21,3414.54,1480648.51,24585563.45,2.34


In [5]:
hs300["交易日期"] = pd.to_datetime(hs300["交易日期"])
hs300_after_2020 = hs300[hs300['交易日期']>=pd.to_datetime("2020-01-02")]

### 获取涨跌幅数据

In [7]:
r = hs300_after_2020['涨跌幅(%)'].tolist()
r = [x / 100 for x in r]
N = len(r)

In [9]:
print(r,N)

[0.013600000000000001, -0.0018, -0.0038, 0.0075, -0.0115, 0.0127, -0.0003, 0.0098, -0.0034000000000000002, -0.0055000000000000005, -0.0042, 0.0014000000000000002, 0.0075, -0.0171, 0.0043, -0.031, -0.0788, 0.0264, 0.0113, 0.018600000000000002, 0.0, 0.0040999999999999995, 0.009300000000000001, 0.008100000000000001, -0.0062, 0.006999999999999999, 0.0225, -0.0049, -0.0015, 0.023, 0.0012, -0.004, -0.0022, -0.0123, 0.0029, -0.0355, 0.0329, 0.0053, 0.0058, 0.0223, -0.016200000000000003, -0.0342, 0.021400000000000002, -0.013300000000000001, -0.0192, -0.0141, -0.043, -0.0049, -0.019799999999999998, -0.013000000000000001, 0.0179, -0.0336, 0.0269, 0.0269, -0.0066, 0.0032, -0.0097, 0.0033, -0.003, 0.016200000000000003, -0.005699999999999999, 0.022799999999999997, -0.004699999999999999, 0.0033, -0.0062, -0.0042, 0.019299999999999998, -0.0074, 0.0013, 0.0098, 0.0036, -0.0118, 0.008199999999999999, -0.0025, -0.0086, 0.0068000000000000005, 0.0069, 0.0046, 0.0118, 0.0060999999999999995, -0.0029, 0.0098

In [11]:
# 初始化DP表
dp = np.zeros((N+1, 101))  # dp[t][s]表示第t天仓位为s时的最大累计收益
dp[0, 0] = 1.0
path = np.zeros((N+1, 101), dtype=int)  # 路径回溯

### 遍历得到最优解

In [13]:
# 动态规划求解
for t in range(1, N+1):
    for s_prev in range(101):
        if dp[t-1, s_prev] == 0:
            continue
        delta_min = max(-10, -s_prev)
        delta_max = min(10, 100 - s_prev)
        for delta in range(delta_min, delta_max + 1):
            s_new = s_prev + delta
            if s_new < 0 or s_new > 100:
                continue
            daily_return = (s_prev / 100) * r[t-1]
            new_value = dp[t-1, s_prev] * (1 + daily_return)
            if new_value > dp[t, s_new]:
                dp[t, s_new] = new_value
                path[t, s_new] = s_prev

In [15]:
# 回溯最优路径
s_optimal = np.zeros(N+1, dtype=int)
s_optimal[N] = np.argmax(dp[N])
for t in range(N, 0, -1):
    s_optimal[t-1] = path[t, s_optimal[t]]

print(s_optimal[N])

90


In [17]:
# 计算评估指标
value = np.zeros(N+1)
value[0] = 1.0
for t in range(1, N+1):
    value[t] = value[t-1] * (1 + (s_optimal[t-1] / 100) * r[t-1])

cumulative_return = value[-1] - 1
peak = 1.0
max_drawdown = 0.0
for t in range(1, N+1):
    if value[t] > peak:
        peak = value[t]
    else:
        drawdown = (peak - value[t]) / peak
        max_drawdown = max(max_drawdown, drawdown)

daily_returns = [(s_optimal[t-1] / 100) * r[t-1] for t in range(1, N+1)]
sharpe_ratio = np.mean(daily_returns) / np.std(daily_returns) if np.std(daily_returns) != 0 else 0

print(f"累计收益: {cumulative_return:.4f}")
print(f"最大回撤: {max_drawdown:.4f}")
print(f"夏普比率: {sharpe_ratio:.4f}")

累计收益: 3.5107
最大回撤: 0.0287
夏普比率: 0.2400


In [20]:
value

array([1.        , 1.        , 0.99982   , 0.99906014, 1.00130802,
       0.99670201, 1.00303106, 1.0029107 , 1.00585926, 1.00517527,
       1.00462243, 1.00462243, 1.00476307, 1.00551665, 1.00551665,
       1.00594902, 1.00594902, 1.00594902, 1.00860472, 1.01088417,
       1.0165249 , 1.0165249 , 1.01860878, 1.02429262, 1.03010036,
       1.02499106, 1.0314485 , 1.05465609, 1.05000506, 1.04858755,
       1.07270507, 1.07386359, 1.07042722, 1.06877877, 1.06089118,
       1.06242947, 1.04734297, 1.06457176, 1.06682866, 1.06868494,
       1.07345127, 1.07171228, 1.07171228, 1.07400575, 1.07400575,
       1.07400575, 1.07400575, 1.07400575, 1.07400575, 1.07187922,
       1.06909233, 1.07483336, 1.06038759, 1.07464981, 1.09199466,
       1.08839107, 1.09048078, 1.08307642, 1.08593574, 1.08300372,
       1.10054838, 1.09490256, 1.11986634, 1.11512931, 1.11844124,
       1.11289377, 1.10868703, 1.13008469, 1.12255833, 1.12401765,
       1.13503303, 1.13871053, 1.12796111, 1.13628546, 1.13401